In [1]:
import datetime, time, random, string

def one_station(name):
    # temp pattern
    month_avg = [27,31,44,58,70,79,83,81,74,61,46,32]
    shift = (random.random()-0.5) * 30
    month_avg = [m + shift + (random.random()-0.5) * 5 for m in month_avg]
    
    # rain pattern
    start_rain = [0.1,0.1,0.3,0.5,0.4,0.2,0.2,0.1,0.2,0.2,0.2,0.1]
    shift = (random.random()-0.5) * 0.1
    start_rain = [r + shift + (random.random() - 0.5) * 0.2 for r in start_rain]
    stop_rain = 0.2 + random.random() * 0.2

    # day's state
    today = datetime.date(2000, 1, 1)
    temp = month_avg[0]
    raining = False
    
    # gen weather
    while True:
        # choose temp+rain
        month = today.month - 1
        temp = temp * 0.8 + month_avg[month] * 0.2 + (random.random()-0.5) * 20
        if temp < 32:
            raining=False
        elif raining and random.random() < stop_rain:
            raining = False
        elif not raining and random.random() < start_rain[month]:
            raining = True

        yield (today.strftime("%Y-%m-%d"), name, temp, raining)

        # next day
        today += datetime.timedelta(days=1)
        
def all_stations(count=10, sleep_sec=1):
    assert count <= 26
    stations = []
    for name in string.ascii_uppercase[:count]:
        stations.append(one_station(name))
    while True:
        for station in stations:
            yield next(station)
        time.sleep(sleep_sec)

In [2]:
# loops forever because the weather never ends...
for row in all_stations(3):
    print(row) # date, station, temp, raining

('2000-01-01', 'A', 14.624324696251467, False)
('2000-01-01', 'B', 28.569073444183076, False)
('2000-01-01', 'C', 24.8666061057328, False)
('2000-01-02', 'A', 17.59238292310423, False)
('2000-01-02', 'B', 25.962002194419092, False)
('2000-01-02', 'C', 31.27538841258746, False)
('2000-01-03', 'A', 18.38805119134942, False)
('2000-01-03', 'B', 18.894169876477296, False)
('2000-01-03', 'C', 31.73930889799926, False)
('2000-01-04', 'A', 25.376209017982887, False)
('2000-01-04', 'B', 16.825901727030388, False)
('2000-01-04', 'C', 34.660139426896805, False)
('2000-01-05', 'A', 33.23279847601561, False)
('2000-01-05', 'B', 13.971304204995596, False)
('2000-01-05', 'C', 27.05587204717956, False)


KeyboardInterrupt: 

In [3]:
from kafka import KafkaAdminClient, KafkaProducer, KafkaConsumer, TopicPartition
from kafka.admin import NewTopic
from kafka.errors import TopicAlreadyExistsError, UnknownTopicOrPartitionError

broker = "kafka:9092"
admin = KafkaAdminClient(bootstrap_servers =[broker])

In [4]:
from kafka import KafkaAdminClient, KafkaProducer, KafkaConsumer, TopicPartition
from kafka.admin import NewTopic
from kafka.errors import TopicAlreadyExistsError, UnknownTopicOrPartitionError

broker = "kafka:9092"
admin = KafkaAdminClient(bootstrap_servers =[broker])
try:
    admin.delete_topics(["stations", "stations-json"])
    print("deleted")
except UnknownTopicOrPartitionError:
    print("cannot delete (may not exist yet)")

time.sleep(1)
admin.create_topics([NewTopic("stations", 6, 1)])
admin.create_topics([NewTopic("stations-json", 6, 1)])
admin.list_topics()

cannot delete (may not exist yet)


['stations-json', 'stations']

In [8]:
!python3 -m grpc_tools.protoc -I=. --python_out=. --grpc_python_out=. message.proto